In [ ]:
#Method 2
import cv2
import numpy as np
from scipy.interpolate import splprep, splev
import os
import random

# Directories (New folder for Method 2)
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/image_method2",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/annotation_method2"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

pore_points = np.array([
    [50, 40], [48, 58], [47, 72], [45, 85], [44, 98], [46, 113],
    [58, 123], [54, 97], [60, 95], [85, 78], [88, 60]
])

def generate_pore_shape(scale=1.0):
    pts = pore_points * scale
    tck, _ = splprep(pts.T, s=0.5, per=True)
    u = np.linspace(0, 1.0, 300)
    curve = np.stack(splev(u, tck), axis=-1).astype(np.int32)
    return curve

def draw_pore_soft_mask(image, center, scale=0.4):
    h, w = image.shape[:2]
    shape = generate_pore_shape(scale)
    shape[:, 0] += center[0] - shape[:, 0].mean().astype(int)
    shape[:, 1] += center[1] - shape[:, 1].mean().astype(int)

    # Create alpha mask
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [shape], 255)
    mask_blur = cv2.GaussianBlur(mask, (9, 9), 3)

    # Blend using alpha mask
    mask_blur_float = mask_blur.astype(float) / 255.0
    mask_blur_float = cv2.merge([mask_blur_float]*3)

    black_shape = np.zeros_like(image)
    blended = (mask_blur_float * black_shape + (1 - mask_blur_float) * image).astype(np.uint8)

    return blended, shape

def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 2, x_center, y_center, width, height

for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue
    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(mask, [polygon], 255)

    valid = np.column_stack(np.where(mask == 255))
    if len(valid) == 0:
        continue

    cx, cy = random.choice(valid)
    image, pore_shape = draw_pore_soft_mask(image, (cy, cx))

    xmin, ymin = pore_shape.min(axis=0)
    xmax, ymax = pore_shape.max(axis=0)
    label = to_yolo_bbox(xmin, ymin, xmax, ymax, w, h)

    cv2.imwrite(os.path.join(dirs["output_images_dir"], fname), image)
    with open(os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt")), 'w') as f:
        f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")



In [ ]:
#method 3 improved
import cv2
import numpy as np
from scipy.interpolate import splprep, splev
import os
import random

# Directories
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/image_method3_safe",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/annotation_method3_safe"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

# Pore control points
pore_points = np.array([
    [50, 40], [48, 58], [47, 72], [45, 85], [44, 98], [46, 113],
    [58, 123], [54, 97], [60, 95], [85, 78], [88, 60]
])

def generate_pore_shape(scale=1.0):
    pts = pore_points * scale
    tck, _ = splprep(pts.T, s=0.5, per=True)
    u = np.linspace(0, 1.0, 300)
    curve = np.stack(splev(u, tck), axis=-1).astype(np.int32)
    return curve

def draw_pore_supersample(image, center, scale=0.4, upscale=4):
    h, w = image.shape[:2]
    big_h, big_w = h * upscale, w * upscale
    big_canvas = cv2.resize(image, (big_w, big_h), interpolation=cv2.INTER_LINEAR)

    shape = generate_pore_shape(scale * upscale)
    shape[:, 0] += upscale * center[0] - shape[:, 0].mean().astype(int)
    shape[:, 1] += upscale * center[1] - shape[:, 1].mean().astype(int)

    cv2.fillPoly(big_canvas, [shape], (0, 0, 0))
    result = cv2.resize(big_canvas, (w, h), interpolation=cv2.INTER_AREA)
    shape = (shape / upscale).astype(np.int32)

    return result, shape

def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 2, x_center, y_center, width, height

def get_safe_valid_points(mask, pore_shape_size=50, margin=5):
    # Remove unsafe margin from mask
    h, w = mask.shape
    safe_mask = np.zeros_like(mask)
    safe_mask[margin:h-margin, margin:w-margin] = mask[margin:h-margin, margin:w-margin]

    valid = np.column_stack(np.where(safe_mask == 255))
    return valid

for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue

    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(mask, [polygon], 255)

    valid = get_safe_valid_points(mask)
    if len(valid) == 0:
        print(f"⚠️ No safe ROI in {fname}")
        continue

    cx, cy = random.choice(valid)
    image, pore_shape = draw_pore_supersample(image, (cy, cx))

    xmin, ymin = pore_shape.min(axis=0)
    xmax, ymax = pore_shape.max(axis=0)
    label = to_yolo_bbox(xmin, ymin, xmax, ymax, w, h)

    cv2.imwrite(os.path.join(dirs["output_images_dir"], fname), image)
    with open(os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt")), 'w') as f:
        f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")

    print(f"✅ Saved {fname} with pore (Method 3 + ROI safety)")


✅ Saved 0c9c31fa-8-44.1.jpg with pore (Method 3 + ROI safety)


In [ ]:
#Merho 1
import os
import cv2
import numpy as np
import random
from scipy.interpolate import splprep, splev

# Directories
dirs = {
    "image_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/images",
    "annotation_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/yolov8",
    "output_images_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/image",
    "output_labels_dir": "F:/Pomodoro/Work/TIME/Script/Thesis-Abbas-Segmentation/PolygontoYOLO/ErrorPlayground/pore_dataset/annotation"
}

os.makedirs(dirs["output_images_dir"], exist_ok=True)
os.makedirs(dirs["output_labels_dir"], exist_ok=True)

# Pore shape points (11 points example from your previous finalized shape)
pore_points = np.array([
    [50, 40], [48, 58], [47, 72], [45, 85], [44, 98], [46, 113],
    [58, 123], [54, 97], [60, 95], [85, 78], [88, 60]
])

# Generate smooth pore shape with anti-jagged method (supersampling)
def generate_pore_shape(scale=1.0):
    pts = pore_points * scale
    tck, _ = splprep(pts.T, s=0.5, per=True)
    u = np.linspace(0, 1.0, 300)
    curve = np.stack(splev(u, tck), axis=-1).astype(np.int32)
    return curve

# Draw pore safely inside image using mask check
def draw_pore_on_image_safely(image, mask, scale=0.4):
    h, w = image.shape[:2]
    eroded_mask = cv2.erode(mask, np.ones((10, 10), np.uint8), iterations=1)
    valid_points = np.column_stack(np.where(eroded_mask == 255))

    random.shuffle(valid_points)  # shuffle to avoid infinite loops
    for attempt, (cy, cx) in enumerate(valid_points):
        # Generate shape
        shape = generate_pore_shape(scale)
        shape[:, 0] += cx - shape[:, 0].mean().astype(int)
        shape[:, 1] += cy - shape[:, 1].mean().astype(int)

        # Create pore mask to check intersection
        temp_mask = np.zeros_like(mask)
        cv2.fillPoly(temp_mask, [shape], 255)
        intersection = cv2.bitwise_and(eroded_mask, temp_mask)

        if cv2.countNonZero(intersection) == cv2.countNonZero(temp_mask):
            # Valid pore found fully inside ROI
            # Supersampling approach to reduce jaggedness
            factor = 4
            upscale = cv2.resize(image, (w * factor, h * factor), interpolation=cv2.INTER_CUBIC)
            upscale_shape = (shape * factor).astype(np.int32)
            cv2.fillPoly(upscale, [upscale_shape], (0, 0, 0))
            downscale = cv2.resize(upscale, (w, h), interpolation=cv2.INTER_AREA)
            np.copyto(image, downscale)
            return shape
    print("⚠️ No valid placement found")
    return None

# Convert bbox to YOLO format
def to_yolo_bbox(xmin, ymin, xmax, ymax, img_w, img_h):
    x_center = (xmin + xmax) / 2 / img_w
    y_center = (ymin + ymax) / 2 / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return 2, x_center, y_center, width, height  # class_id=2

# Main loop
for fname in os.listdir(dirs["image_dir"]):
    if not fname.endswith(".jpg"):
        continue
    img_path = os.path.join(dirs["image_dir"], fname)
    ann_path = os.path.join(dirs["annotation_dir"], fname.replace(".jpg", ".txt"))
    if not os.path.exists(ann_path):
        print(f"⚠️ Missing annotation for {fname}")
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    with open(ann_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != 3:
                continue
            coords = np.array(parts[1:], dtype=float).reshape(-1, 2)
            coords[:, 0] *= w
            coords[:, 1] *= h
            polygon = coords.astype(np.int32)
            cv2.fillPoly(mask, [polygon], 255)

    if np.count_nonzero(mask) == 0:
        print(f"⚠️ No valid Class 3 ROI in {fname}")
        continue

    pore_shape = draw_pore_on_image_safely(image, mask)
    if pore_shape is None:
        continue  # Skip if no safe placement found

    xmin, ymin = pore_shape.min(axis=0)
    xmax, ymax = pore_shape.max(axis=0)
    label = to_yolo_bbox(xmin, ymin, xmax, ymax, w, h)

    out_img_path = os.path.join(dirs["output_images_dir"], fname)
    out_lbl_path = os.path.join(dirs["output_labels_dir"], fname.replace(".jpg", ".txt"))
    cv2.imwrite(out_img_path, image)
    with open(out_lbl_path, 'w') as f:
        f.write(f"{label[0]} {label[1]:.6f} {label[2]:.6f} {label[3]:.6f} {label[4]:.6f}\n")

    print(f"✅ Saved {fname} with safe pore inside ROI")


✅ Saved 0c9c31fa-8-44.1.jpg with safe pore inside ROI
